In [4]:
import os
import monai
from monai.transforms import (
    Compose,
    LoadImaged,
    EnsureChannelFirstd,
    ScaleIntensityd,
    ToTensord,
)
from monai.data import CacheDataset, DataLoader
from monai.networks.nets import UNet
from monai.networks.layers import Norm
from monai.losses import DiceLoss
from monai.metrics import DiceMetric
from monai.optimizers import Novograd
from monai.inferers import sliding_window_inference
import torch

# Set up the directory paths for images and labels
base_dir = "./datasets/data/datasets_unet/"
image_dir_train = os.path.join(base_dir, "train/images")
label_dir_train = os.path.join(base_dir, "train/masks")
image_dir_val = os.path.join(base_dir, "val/images")
label_dir_val = os.path.join(base_dir, "val/masks")
# image_dir_test = os.path.join(base_dir, "test/images")
# label_dir_test = os.path.join(base_dir, "test/masks")

# Get the list of image and label files
images_train = sorted([os.path.join(image_dir_train, f) for f in os.listdir(image_dir_train) if f.endswith('.png')])
labels_train = sorted([os.path.join(label_dir_train, f) for f in os.listdir(label_dir_train) if f.endswith('.png')])
images_val = sorted([os.path.join(image_dir_val, f) for f in os.listdir(image_dir_val) if f.endswith('.png')])
labels_val = sorted([os.path.join(label_dir_val, f) for f in os.listdir(label_dir_val) if f.endswith('.png')])
# images_test = sorted([os.path.join(image_dir_test, f) for f in os.listdir(image_dir_test) if f.endswith('.png')])
# labels_test = sorted([os.path.join(label_dir_test, f) for f in os.listdir(label_dir_test) if f.endswith('.png')])

# Create a list of dictionaries for paired image and label paths
data_dicts_train = [{'image': image_name, 'label': label_name} for image_name, label_name in zip(images_train, labels_train)]
data_dicts_val = [{'image': image_name, 'label': label_name} for image_name, label_name in zip(images_val, labels_val)]
# data_dicts_test = [{'image': image_name, 'label': label_name} for image_name, label_name in zip(images_test, labels_test)]

# Define the transformations to preprocess the images
train_transforms = Compose([
    LoadImaged(keys=['image', 'label']),
    EnsureChannelFirstd(keys=['image', 'label']),
    ScaleIntensityd(keys=['image', 'label']),
    ToTensord(keys=['image', 'label']),
])
val_transforms = Compose([
    LoadImaged(keys=['image', 'label']),
    EnsureChannelFirstd(keys=['image', 'label']),
    ScaleIntensityd(keys=['image', 'label']),
    ToTensord(keys=['image', 'label']),
])
# test_transforms = Compose([
#     LoadImaged(keys=['image', 'label']),
#     EnsureChannelFirstd(keys=['image', 'label']),
#     ScaleIntensityd(keys=['image', 'label']),
#     ToTensord(keys=['image', 'label']),
# ])

# Create a MONAI dataset and a data loader for training
train_ds = CacheDataset(data=data_dicts_train, transform=train_transforms, cache_rate=1.0)
train_loader = DataLoader(train_ds, batch_size=2, shuffle=True)
val_ds = CacheDataset(data=data_dicts_val, transform=val_transforms, cache_rate=1.0)
val_loader = DataLoader(val_ds, batch_size=2, shuffle=True)
# test_ds = CacheDataset(data=data_dicts_test, transform=test_transforms, cache_rate=1.0)
# test_loader = DataLoader(test_ds, batch_size=2, shuffle=True)

Loading dataset: 100%|██████████| 783/783 [00:07<00:00, 103.24it/s]


In [5]:
import torch
from tqdm import tqdm
import os
from ignite.engine import Engine, Events
from ignite.handlers import EarlyStopping, ModelCheckpoint
from monai.networks.nets import UNet
from monai.transforms import (
    Compose, LoadImaged, EnsureChannelFirstd, ScaleIntensityd, ToTensord
)
from monai.data import CacheDataset, DataLoader
from monai.losses import DiceLoss
from monai.metrics import DiceMetric


# Set up the U-Net model
model = UNet(
    spatial_dims=2,  # Use `spatial_dims` instead of `dimensions` if needed
    in_channels=1,
    out_channels=1, # If your dataset has binary segmentation, you need only 1 output channel
    channels=(16, 32, 64, 128, 256),
    strides=(2, 2, 2, 2),
    num_res_units=2,
    norm=Norm.BATCH,
)
# After initializing the model, move it to the GPU
# model = model.cuda()


# Set up the loss function and optimizer
loss_function = DiceLoss(to_onehot_y=False, sigmoid=True)
optimizer = Novograd(model.parameters(), lr=0.001)

num_epochs = 5
model_dir = './models'
os.makedirs(model_dir, exist_ok=True)

# Assuming model, train_loader, val_loader, optimizer, and loss_function are already defined

# Define the training step
def train_step(engine, batch):
    model.train()
    optimizer.zero_grad()
    inputs, labels = batch['image'], batch['label']
    outputs = model(inputs)
    loss = loss_function(outputs, labels)
    loss.backward()
    optimizer.step()
    return loss.item()

trainer = Engine(train_step)

# Define the validation step
def validation_step(engine, batch):
    model.eval()
    with torch.no_grad():
        inputs, labels = batch['image'], batch['label']
        outputs = model(inputs)
        loss = loss_function(outputs, labels)
    return loss.item()

evaluator = Engine(validation_step)

# Define the score function for early stopping
def score_function(engine):
    return -engine.state.metrics['average_loss']

# Attach the validation metric (loss) to the evaluator
from ignite.metrics import RunningAverage
RunningAverage(output_transform=lambda x: x).attach(evaluator, 'average_loss')

# Setup model checkpoint to save the best model based on validation loss
# checkpoint_handler = ModelCheckpoint(model_dir, 'checkpoint', n_saved=1, create_dir=True, score_function=score_function, score_name='val_loss', global_step_transform=global_step_from_engine(trainer))
checkpoint_handler = ModelCheckpoint(model_dir, 'checkpoint', n_saved=1, create_dir=True, score_function=score_function, score_name='val_loss')

# evaluator.add_event_handler(Events.COMPLETED, checkpoint_handler, {'model': model})
evaluator.add_event_handler(Events.COMPLETED, checkpoint_handler, {'model': model})

# Setup early stopping
early_stopper = EarlyStopping(patience=10, score_function=score_function, trainer=trainer)
evaluator.add_event_handler(Events.COMPLETED, early_stopper)

@trainer.on(Events.EPOCH_COMPLETED)
def run_validation(engine):
    evaluator.run(val_loader)

trainer.run(train_loader, max_epochs=num_epochs)

# The best model is saved with the prefix 'checkpoint' in the directory './models'
# You can rename it here if you like
best_model_path = os.path.join(model_dir, 'best_model.pth')
os.rename(checkpoint_handler.last_checkpoint, best_model_path)
print(f"The best model has been saved as {best_model_path}")


AttributeError: partially initialized module 'torch._dynamo' has no attribute 'external_utils' (most likely due to a circular import)

In [ ]:
import torch
from tqdm import tqdm
import os
from ignite.engine import Engine, Events
from ignite.handlers import EarlyStopping, ModelCheckpoint
from monai.networks.nets import UNet
from monai.transforms import (
    Compose, LoadImaged, EnsureChannelFirstd, ScaleIntensityd, ToTensord
)
from monai.data import CacheDataset, DataLoader
from monai.losses import DiceLoss
from monai.metrics import DiceMetric

num_epochs = 5
model_dir = './models'
os.makedirs(model_dir, exist_ok=True)

model_path = os.path.join(model_dir, 'best_model.pth')

# Initialize early stopping
early_stopping = EarlyStopping(patience=10, score_function=lambda x: -x, saver=CheckpointSaver(save_dir=model_dir, save_dict={'model': model}, save_key='val_loss'))

# Initialize best validation loss for comparison
best_val_loss = float('inf')

for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    for batch_data in train_loader:
        inputs, labels = batch_data['image'], batch_data['label']
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    
    # Validation phase
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for batch_data in val_loader:
            inputs, labels = batch_data['image'], batch_data['label']
            outputs = model(inputs)
            loss = loss_function(outputs, labels)
            val_loss += loss.item()

    # Calculate average losses
    train_loss /= len(train_loader)
    val_loss /= len(val_loader)
    print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}')

    # Check if this is the best model (based on validation loss)
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), model_path)  # Save the best model
        
    # Early stopping check
    early_stopping(epoch, val_loss)
    if early_stopping.early_stop:
        print("Early stopping triggered")
        break

# After training, you might want to rename the best model
final_model_path = os.path.join(model_dir, 'final_best_model.pth')
os.rename(model_path, final_model_path)
print(f"The best model has been saved as {final_model_path}")











# Define the training step
def train_step(engine, batch):
    model.train()
    optimizer.zero_grad()
    inputs, labels = batch['image'], batch['label']
    outputs = model(inputs)
    loss = loss_function(outputs, labels)
    loss.backward()
    optimizer.step()
    return loss.item()

trainer = Engine(train_step)

# Define the validation step
def validation_step(engine, batch):
    model.eval()
    with torch.no_grad():
        inputs, labels = batch['image'], batch['label']
        outputs = model(inputs)
        loss = loss_function(outputs, labels)
    return loss.item()

evaluator = Engine(validation_step)

# Define the score function for early stopping
def score_function(engine):
    return -engine.state.metrics['average_loss']

# Attach the validation metric (loss) to the evaluator
from ignite.metrics import RunningAverage
RunningAverage(output_transform=lambda x: x).attach(evaluator, 'average_loss')

# Setup model checkpoint to save the best model based on validation loss
checkpoint_handler = ModelCheckpoint(model_dir, 'checkpoint', n_saved=1, create_dir=True, score_function=score_function, score_name='val_loss', global_step_transform=global_step_from_engine(trainer))
evaluator.add_event_handler(Events.COMPLETED, checkpoint_handler, {'model': model})

# Setup early stopping
early_stopper = EarlyStopping(patience=10, score_function=score_function, trainer=trainer)
evaluator.add_event_handler(Events.COMPLETED, early_stopper)

@trainer.on(Events.EPOCH_COMPLETED)
def run_validation(engine):
    evaluator.run(val_loader)

trainer.run(train_loader, max_epochs=num_epochs)

# The best model is saved with the prefix 'checkpoint' in the directory './models'
# You can rename it here if you like
best_model_path = os.path.join(model_dir, 'best_model.pth')
os.rename(checkpoint_handler.last_checkpoint, best_model_path)
print(f"The best model has been saved as {best_model_path}")


NameError: name 'CheckpointSaver' is not defined

In [3]:
import os
import torch
from ignite.engine import Engine, Events
from ignite.handlers import EarlyStopping
from monai.networks.nets import UNet
from monai.transforms import (
    Compose, LoadImaged, EnsureChannelFirstd, ScaleIntensityd, ToTensord
)
from monai.data import CacheDataset, DataLoader
from monai.losses import DiceLoss
from monai.metrics import DiceMetric


# Training step function
def train_step(engine, batch):
    model.train()
    optimizer.zero_grad()
    inputs, labels = batch['image'], batch['label']
    outputs = model(inputs)
    loss = loss_function(outputs, labels)
    loss.backward()
    optimizer.step()
    return loss.item()

# Validation step function
def validation_step(engine, batch):
    model.eval()
    with torch.no_grad():
        inputs, labels = batch['image'], batch['label']
        outputs = model(inputs)
        return outputs, labels

# Score function for early stopping
def score_function(engine):
    val_loss = engine.state.metrics['average_loss']
    return -val_loss

trainer = Engine(train_step)
evaluator = Engine(validation_step)

# Attach metrics to the evaluator
val_metrics = {
    'average_loss': Loss(loss_function),
    'dice_metric': DiceMetric(include_background=True, reduction="mean")
}
for name, metric in val_metrics.items():
    metric.attach(evaluator, name)

# EarlyStopping handler to stop training after no improvement
handler = EarlyStopping(patience=10, score_function=score_function, trainer=trainer)
evaluator.add_event_handler(Events.COMPLETED, handler)

# Function to run the evaluator on the validation dataset
@trainer.on(Events.EPOCH_COMPLETED)
def run_validation(engine):
    evaluator.run(val_loader)

# Function to save the model after each epoch
best_val_loss = float('inf')
model_dir = './models'
os.makedirs(model_dir, exist_ok=True)

@trainer.on(Events.EPOCH_COMPLETED)
def save_best_model(engine):
    global best_val_loss
    current_val_loss = evaluator.state.metrics['average_loss']
    if current_val_loss < best_val_loss:
        best_val_loss = current_val_loss
        torch.save(model.state_dict(), os.path.join(model_dir, 'best_model.pth'))

# Running the training process
trainer.run(train_loader, max_epochs=100)

# Optional: Rename the best model at the end
final_model_path = os.path.join(model_dir, 'final_best_model.pth')
os.rename(os.path.join(model_dir, 'best_model.pth'), final_model_path)
print(f"The best model has been saved as {final_model_path}")




KeyboardInterrupt

